# Autocombat TOP and StrokeMRI vs. HELIUS  harmonized datasets

Note this must be run in the `comscan6` environment

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

sys.path.insert(0, '../../') # path to functions

import cvasl.harmony as har
import cvasl.seperated as sep
import cvasl.vendor.comscan.neurocombat as autocombat

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
# Datasets for this work
HELIUS = pd.read_csv('../new_data/TrainingDataComplete_HELIUS.csv')
MRI = pd.read_csv('../new_data/TrainingDataComplete_StrokeMRI.csv')
TOP = pd.read_csv('../new_data/TrainingDataComplete_TOP.csv')

In [ ]:
HELIUS.Sex.unique()

In [ ]:
TOP.Sex.unique()

In [ ]:
MRI.Sex.unique()

In [ ]:
TOP = TOP.drop([ 'ID'], axis= 1)
TOP.head(3)

In [ ]:
MRI = MRI.drop(['ID'],axis = 1)
MRI.tail(5)

In [ ]:
HELIUS = HELIUS.drop(['ID'],axis = 1)
HELIUS.tail(5)

In [ ]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI['Site'] = 0
TOPMRI.head(3) 

In [ ]:
datasets = [TOPMRI, HELIUS]
new_frames = sep.deal_with_readout_and_labelling(datasets, ['M0'])

In [ ]:
# make sure things are inproper order
for frame in new_frames:
    print(len(frame))
print(len(TOPMRI))
print(len(HELIUS))

In [ ]:
#new_frames[0]
TOPMRI = new_frames[0]
HELIUS = new_frames[1]


In [ ]:
new_frame_datasets=  [TOPMRI,  HELIUS, ]
for everyone in new_frame_datasets:
    everyone.columns =  everyone.columns.str.lower()
TOPMRI.head(3)

In [ ]:
TOPMRIHELIUS = pd.concat([TOPMRI, HELIUS])
TOPMRIHELIUS

In [ ]:
TOPMRIHELIUS['decade']=(TOPMRIHELIUS['age']/10).round()
#TOPMRI['decade']

In [ ]:
TOPMRIHELIUS = TOPMRIHELIUS.sort_values(by='age')
TOPMRIHELIUS.reset_index(inplace=True)
TOPMRIHELIUS['fine_grain'] = TOPMRIHELIUS['age'].rolling(2).sum()/2

In [ ]:
TOPMRIHELIUS[2:].fine_grain.iloc[::2] = TOPMRIHELIUS[:].fine_grain.iloc[1::2]
TOPMRIHELIUS['fine_grain']

In [ ]:
TOPMRIHELIUS['fine_grain'][0] = TOPMRIHELIUS['fine_grain'][1]
TOPMRIHELIUS['fine_grain']

In [ ]:
TOPMRIHELIUS['decade'].unique()

In [ ]:
TOPMRIHELIUS.columns

In [ ]:
combat = autocombat.Combat(
    features=[ 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
        'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
        'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',
             'ld', 'pld', 'labelling', 'readout', ],
    sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['decade'],)

fg_combat = autocombat.Combat(
    features=[ 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
        'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
        'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',
             'ld', 'pld', 'labelling', 'readout', ],
    sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['fine_grain'],)


In [ ]:
print(combat.fit(TOPMRIHELIUS))
print(fg_combat.fit(TOPMRIHELIUS))

In [ ]:
print(combat.gamma_star_)
print(fg_combat.gamma_star_)

In [ ]:
transformed_TOPMRIHELIUS= combat.transform(TOPMRIHELIUS)
transformed_TOPMRIHELIUS.head(3)

In [ ]:
fg_transformed_TOPMRIHELIUS= fg_combat.transform(TOPMRIHELIUS)
fg_transformed_TOPMRIHELIUS.head(3)

In [ ]:
TOPMRIHELIUS.head(3)

In [ ]:
transformed_TOPMRIHELIUS= combat.transform(TOPMRIHELIUS)
transformed_TOPMRIHELIUS.head(3)

In [ ]:
fg_transformed_TOPMRIHELIUS= fg_combat.transform(TOPMRIHELIUS)
fg_transformed_TOPMRIHELIUS.head(3)

In [ ]:
transformed_TOPMRIHELIUS.columns

In [ ]:
TOPMRI_transformed = transformed_TOPMRIHELIUS[ transformed_TOPMRIHELIUS['site']==0]
TOPMRI_transformed = TOPMRI_transformed.drop(['site', 'decade','fine_grain', 'index'], axis=1)
TOPMRI_transformed.head(3)

In [ ]:
fg_TOPMRI_transformed = fg_transformed_TOPMRIHELIUS[ fg_transformed_TOPMRIHELIUS['site']==0]
fg_TOPMRI_transformed = fg_TOPMRI_transformed .drop(['site', 'decade','fine_grain', 'index'], axis=1)
fg_TOPMRI_transformed.head(3)

In [ ]:
HELIUS_transformed = transformed_TOPMRIHELIUS[ transformed_TOPMRIHELIUS['site']==1]
HELIUS_transformed = HELIUS_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
HELIUS_transformed.head(3)

In [ ]:
fg_HELIUS_transformed = fg_transformed_TOPMRIHELIUS[fg_transformed_TOPMRIHELIUS['site']==1]
fg_HELIUS_transformed = fg_HELIUS_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
fg_HELIUS_transformed.head(3)

In [ ]:
TOPMRI_transformed.to_csv('harm_results/autocombat/autocom_harm_topmri_v_h.csv')
HELIUS_transformed.to_csv('harm_results/autocombat/autocom_harm_HELIUS.csv')

fg_TOPMRI_transformed.to_csv('harm_results/autocombat/fg_autocom_harm_topmri_v_h.csv')
fg_HELIUS_transformed.to_csv('harm_results/autocombat/fg_autocom_harm_HELIUS1.csv')

# Data quality check

In [ ]:
TOPMRI_transformed.isna().sum().sum()

In [ ]:
HELIUS_transformed.isna().sum().sum()

In [ ]:
number_columns = ['sex', 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',
                 'ld', 'pld', 'labelling', 'readout', ]

In [ ]:
#neuro_harm_top.head(3)
(TOPMRI_transformed[number_columns] < 0).sum()

In [ ]:
(HELIUS_transformed[number_columns] < 0).sum()

In [ ]:
HELIUS_transformed

## So some variables are again problematic, becuas they are negative

 We will wait about logging it until we see other columns we may want to log across all datasets